In [14]:
import pandas as pd
import re
def number_to_chinese_number(num, flag='complex', financial='no'):
    """
    功能：阿拉伯数字转中文数字;
    参数：num Type int or str为输入阿拉伯数字;
          flag 为输出方式：'complex'为输出带单位中文数字，不能超过10亿；
                           'simple'为不带单位中文数字，可以任意大；
                           'week' 为星期中文数字，范围0-7；0和7认为是周日。
          financial str 是否开启财务计算，num可以为小数。
    返回：chinese_number，Type str。
    """
    chinese_number = ''
    if financial == 'no':
        num_dict = {'1': '一', '2': '二', '3': '三', '4': '四', '5': '五', '6': '六', '7': '七', '8': '八', '9': '九',
                    '0': '〇', }
        nums = list(str(num))

        if flag == 'complex':
            index_dict = {1: '', 2: '十', 3: '百', 4: '千', 5: '万', 6: '十', 7: '百', 8: '千', 9: '亿', 10: '十',
                          11: '百', 12: '千', 13: '万', 14: '亿', 15: '十', 16: '百', 17: '千', 18: '万', 19: '亿'}
            try:
                nums_index = [x for x in range(1, len(nums) + 1)][-1::-1]

                for index, item in enumerate(nums):
                    chinese_number = "".join((chinese_number, num_dict[item], index_dict[nums_index[index]]))
                chinese_number = re.sub("〇[十百千〇]*", "〇", chinese_number)
                chinese_number = re.sub("〇亿", "亿", chinese_number)
                chinese_number = re.sub("〇万", "万", chinese_number)
                chinese_number = re.sub("亿万", "亿〇", chinese_number)
                chinese_number = re.sub("〇〇", "〇", chinese_number)
                if chinese_number != '〇':
                    chinese_number = re.sub("〇\\b", "", chinese_number)
            except Exception as e:
                print(e)
                return '输入不能大于亿亿亿，期待下次改进。'

        elif flag == 'simple':
            chinese_number_list = list(map(lambda x: num_dict[x], nums))
            chinese_number = ''.join(chinese_number_list)
        elif flag == 'week':
            num_dict = {'0': '日', '1': '一', '2': '二', '3': '三', '4': '四', '5': '五', '6': '六', '7': '日'}
            if int(num) <= 7:
                num_str = str(num)
                chinese_number = num_dict[num_str]
            else:
                chinese_number = None
    elif financial == 'yes':
        number_arab = '0123456789'
        number_str = '零壹贰叁肆伍陆柒捌玖'
        # 数值字典numDic,和阿拉伯数字是简单的一一对应关系
        number_dict = dict(zip(number_arab, number_str))

        unit_arab = (-2, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
        unit_str = '角分元拾佰仟万亿拾佰仟万亿拾佰仟万亿'
        unit_dict = dict(zip(unit_arab, unit_str))
        number_str = str(num)
        integer_part_number = ''
        decimal_part_number = ''
        try:
            if '.' in number_str:
                integer_part, decimal_part = number_str.split('.')
            else:
                integer_part = number_str
                decimal_part = '0'
            decimal_part = decimal_part[0:2]
            integer_part = list(integer_part)

            decimal_part = list(decimal_part)

            nums_index = [x for x in range(1, len(integer_part) + 1)][-1::-1]
            for index, item in enumerate(integer_part):
                integer_part_number = "".join((integer_part_number, number_dict[item], unit_dict[nums_index[index]]))
            integer_part_number = re.sub("零[拾佰仟零]*", "零", integer_part_number)
            integer_part_number = re.sub("零亿", "亿", integer_part_number)
            integer_part_number = re.sub("零万", "万", integer_part_number)
            integer_part_number = re.sub("亿万", "亿零", integer_part_number)
            integer_part_number = re.sub("零零", "零", integer_part_number)
            integer_part_number = re.sub("零元", "元", integer_part_number)
            if integer_part_number != '零':
                integer_part_number = re.sub("零\\b", "", integer_part_number)

            if len(decimal_part) == 1:
                decimal_part_number = number_dict[decimal_part[0]] + '角'
                decimal_part_number = re.sub("零角", "整", decimal_part_number)
            elif len(decimal_part) == 2:

                nums_index = [x for x in range(1, len(decimal_part) + 1)][-1::-1]
                for index, item in enumerate(decimal_part):
                    decimal_part_number = "".join(
                        (decimal_part_number, number_dict[item], unit_dict[-nums_index[index]]))
                decimal_part_number = re.sub("零角", "零", decimal_part_number)
                decimal_part_number = re.sub("零分", "整", decimal_part_number)
            chinese_number = integer_part_number + decimal_part_number
        except Exception as e:
            print(str(e))
            return '输入不能大于亿亿亿，期待下次改进。'
    return chinese_number


def df_iter(df, num=True):
    df_text = []
    j = 1
    for row in df.iterrows():
        x = []
        for i in range(len(row[1])):
            if num:
                x.append(row[1].index[i] + number_to_chinese_number(j) + '：' + str(row[1][row[1].index[i]]) + '\n')
            else:
                x.append(row[1].index[i] + '：' + str(row[1][row[1].index[i]]) + '\n')
        df_text.append(''.join(x))
        j += 1
    return df_text
df=pd.DataFrame(
                [
                    ['A','1',5,'孔令',2],
                    ['D','1',4,'李黎',2],
                    ['D','1',6,'李黎',2],
                ]
                ,columns=['一','二','三','开始日期','结束日期']
                )
x = df_iter(df)

x

['一一：A\n二一：1\n三一：5\n开始日期一：孔令\n结束日期一：2\n',
 '一二：D\n二二：1\n三二：4\n开始日期二：李黎\n结束日期二：2\n',
 '一三：D\n二三：1\n三三：6\n开始日期三：李黎\n结束日期三：2\n']